# Regressão Linear

In [1]:
!pip install pandas-profiling

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

#### Tratando o Dataframe

In [4]:
df = pd.read_excel('Scout final.xlsx')
df.head()

,Unnamed: 0,Clube,Rodada,Resultado,Posse de Bola,Gols Feitos,Gols Sofridos,Total de Chutes,Faltas Cometidas,Escanteio,Impedimento,Chute ao Gol,Passes Certos,Total de Passes,Defesas,Faltas sofridas,Chutes Errados,Divididas Ganhas
0,0,Santos,1,D,66,0,3,12,9,11,2,4,501,563,0,18,5,9
1,18,Flamengo,1,V,62,1,0,14,25,3,0,10,440,503,5,21,3,9
2,17,Palmeiras,1,D,38,0,1,12,21,4,3,5,244,300,8,25,6,11
3,16,Corinthians,1,D,61,0,1,15,18,10,1,6,431,518,1,14,5,10
4,15,Atlético Goianiense,1,V,40,1,0,7,14,3,5,2,254,351,6,15,3,11


In [5]:
# Utilizando o dummie para transformar a variavel categorica em binaria
df1 = pd.get_dummies(df, columns=['Resultado'])
df1.head()

,Unnamed: 0,Clube,Rodada,Posse de Bola,Gols Feitos,Gols Sofridos,Total de Chutes,Faltas Cometidas,Escanteio,Impedimento,Chute ao Gol,Passes Certos,Total de Passes,Defesas,Faltas sofridas,Chutes Errados,Divididas Ganhas,Resultado_D,Resultado_E,Resultado_V
0,0,Santos,1,66,0,3,12,9,11,2,4,501,563,0,18,5,9,1,0,0
1,18,Flamengo,1,62,1,0,14,25,3,0,10,440,503,5,21,3,9,0,0,1
2,17,Palmeiras,1,38,0,1,12,21,4,3,5,244,300,8,25,6,11,1,0,0
3,16,Corinthians,1,61,0,1,15,18,10,1,6,431,518,1,14,5,10,1,0,0
4,15,Atlético Goianiense,1,40,1,0,7,14,3,5,2,254,351,6,15,3,11,0,0,1


In [15]:
# excluindo as colunas que não serão utilizadas
df2=df1.drop(columns=['Unnamed: 0','Clube','Rodada','Resultado_E','Resultado_D','Gols Feitos','Gols Sofridos'])
df2=df2.rename(columns={'Resultado_V':'Vitória'})
df2

,Posse de Bola,Total de Chutes,Faltas Cometidas,Escanteio,Impedimento,Chute ao Gol,Passes Certos,Total de Passes,Defesas,Faltas sofridas,Chutes Errados,Divididas Ganhas,Vitória
0,66,12,9,11,2,4,501,563,0,18,5,9,0
1,62,14,25,3,0,10,440,503,5,21,3,9,1
2,38,12,21,4,3,5,244,300,8,25,6,11,0
3,61,15,18,10,1,6,431,518,1,14,5,10,0
4,40,7,14,3,5,2,254,351,6,15,3,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,63,12,23,4,3,5,450,530,3,15,5,3,1
698,72,21,6,12,2,8,559,636,2,13,5,2,0
699,62,21,16,11,2,4,593,662,4,8,11,8,0
700,41,4,15,1,2,1,313,387,4,14,3,10,1


In [16]:
# Normalizando os dados

scaler = StandardScaler()

X=df2[['Posse de Bola','Total de Chutes','Escanteio','Total de Passes','Defesas','Divididas Ganhas']]

Y=df2['Vitória']

scaler.fit(X)
X_std = scaler.transform(X)


In [17]:
# Usando o Random Forest para rankear as variaveis
randomForest = RandomForestClassifier(n_estimators=10)
randomForest.fit(X_std, Y)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

print(randomForest.feature_importances_)

[0.17773602 0.1683951  0.13851616 0.25937955 0.10465825 0.15131493]


In [18]:
feature_importances = pd.DataFrame(randomForest.feature_importances_,
                                   index = X.columns,
                                   columns=['Score']).sort_values('Score', ascending=False)
feature_importances

,Score
Total de Passes,0.259380
Posse de Bola,0.177736
Total de Chutes,0.168395
Divididas Ganhas,0.151315
Escanteio,0.138516
Defesas,0.104658


In [19]:
feature_importances.head(3)

,Score
Total de Passes,0.259380
Posse de Bola,0.177736
Total de Chutes,0.168395


## Regressão por Scout

#### Total de Passes

In [20]:
x1=df2[['Posse de Bola','Total de Chutes','Faltas Cometidas','Escanteio','Impedimento','Chute ao Gol','Defesas','Faltas sofridas','Chutes Errados','Divididas Ganhas']]

y1=df2['Total de Passes']

scaler.fit(x1)
x1_std = scaler.transform(x1)


In [23]:
randomForest = RandomForestClassifier(n_estimators=10)
randomForest.fit(x1_std, y1)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

print(randomForest.feature_importances_)

[0.12499518 0.10033228 0.11420032 0.09923936 0.07245932 0.09433055
 0.08635465 0.11440632 0.09060592 0.10307612]


In [24]:
feature_importances1 = pd.DataFrame(randomForest.feature_importances_,
                                   index = x1.columns,
                                   columns=['Score']).sort_values('Score', ascending=False)
feature_importances1

,Score
Posse de Bola,0.124995
Faltas sofridas,0.114406
Faltas Cometidas,0.114200
Divididas Ganhas,0.103076
Total de Chutes,0.100332
Escanteio,0.099239
Chute ao Gol,0.094331
Chutes Errados,0.090606
Defesas,0.086355
Impedimento,0.072459


#### Posse de Bola

In [35]:
x2=df2[['Total de Chutes','Faltas Cometidas','Escanteio','Impedimento','Chute ao Gol','Defesas','Faltas sofridas','Chutes Errados','Divididas Ganhas','Total de Passes']]

y2=df2['Posse de Bola']

scaler.fit(x2)
x2_std = scaler.transform(x2)

In [30]:
# Usando o Random Forest para rankear as variaveis
randomForest = RandomForestClassifier(n_estimators=10)
randomForest.fit(x2_std, y2)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

print(randomForest.feature_importances_)

[0.10046321 0.11207511 0.08842988 0.06142086 0.08324368 0.07985864
 0.11387537 0.08366545 0.10661815 0.17034966]


In [31]:
feature_importances2 = pd.DataFrame(randomForest.feature_importances_,
                                   index = x2.columns,
                                   columns=['Score']).sort_values('Score', ascending=False)
feature_importances2

,Score
Total de Passes,0.170350
Faltas sofridas,0.113875
Faltas Cometidas,0.112075
Divididas Ganhas,0.106618
Total de Chutes,0.100463
Escanteio,0.088430
Chutes Errados,0.083665
Chute ao Gol,0.083244
Defesas,0.079859
Impedimento,0.061421


#### Total de Chutes

In [36]:
x3=df2[['Posse de Bola','Faltas Cometidas','Escanteio','Impedimento','Chute ao Gol','Defesas','Faltas sofridas','Chutes Errados','Divididas Ganhas','Total de Passes']]

y3=df2['Total de Chutes']

scaler.fit(x3)
x3_std = scaler.transform(x3)

In [41]:
# Usando o Random Forest para rankear as variaveis
randomForest = RandomForestClassifier(n_estimators=10)
randomForest.fit(x3_std, y3)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

print(randomForest.feature_importances_)

[0.10775723 0.08840101 0.09126321 0.07580105 0.10726889 0.08260104
 0.11197665 0.12210499 0.09346767 0.11935825]


In [42]:
feature_importances3 = pd.DataFrame(randomForest.feature_importances_,
                                   index = x3.columns,
                                   columns=['Score']).sort_values('Score', ascending=False)
feature_importances3

,Score
Chutes Errados,0.122105
Total de Passes,0.119358
Faltas sofridas,0.111977
Posse de Bola,0.107757
Chute ao Gol,0.107269
Divididas Ganhas,0.093468
Escanteio,0.091263
Faltas Cometidas,0.088401
Defesas,0.082601
Impedimento,0.075801
